In [1]:
from astropy.io import fits
import numpy as np
import pandas as pd

# Catalogue FITS file

Extract targets whose names contain `UDGcand`, assuming they correspond to the `UDGcand_*` targets as used in the `classify-*` files from SpaceFluff. 

---

`@note` _Verify correspondence of names! Don't blindly assume they correspond._

In [2]:
hdul = fits.open('./FDSDWARF_LSB.fits')
header = hdul[0].header
data = hdul[1].data

In [3]:
col_names = data.columns.names
print(col_names)

['target', 'RA', 'DEC', 'PA', 'PA_e', 'arat', 'arat_e', 'r_mag', 'r_mag_e', 'g_mag', 'g_mag_e', 'r_nuc', 'g_nuc', 'reff', 'reff_e', 'n', 'n_e', 'u', 'u_e', 'g', 'g_e', 'r', 'r_e', 'i', 'i_e', 'C', 'RFF', 'Class', 'Ref.']


See http://cdsarc.u-strasbg.fr/ftp/J/A+A/620/A165/ReadMe for description of columns

In [4]:
# among the FITS file targets, there are a number of UDGcand_* entries, 
#   which I assume match UDGCand_* targets as designated in the SpaceFluff .csv files
targets = [d['target'] for d in data if 'UDGcand' in d['target']]

In [5]:
# # save target names to txt file for later comparison to classification votes
# np.savetxt('sf_catalogue_targets.txt', targets, delimiter=',', fmt="%s")

### In notebook `sf_12-04-2021`, I extracted a list of unique target names from `classify-classifications.csv`. 

`@note` _Don't assume the other classification files used the exact same set of targets (presumably so, but verify!)_

In [6]:
candidate_names_classify = np.loadtxt('../SpaceFluff/sf_candidate_names__classification-classify.txt', dtype=str)

- Find the intersection of the targets from the .FITS file and the targets from `classify_classifications.csv`:

In [7]:
intersecting = list(set(targets) & set(candidate_names_classify))
len(intersecting)

238

# FITS file with object data

In [8]:
hdul = fits.open('./LSBS_no_par_sel.fits')
header = hdul[0].header
data = hdul[1].data

In [9]:
print(data.columns.names)

header

['target', 'RA', 'DEC', 'Reff', 'r_mag', 'g_mag', 'axis_ratio', 'pos_angle', 'n', 'u', 'g', 'r', 'i', 'ue', 'ge', 're', 'ie', 'Reffe', 'r_mage', 'ne', 'C', 'mue_r', 'bae', 'RFF']


SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  

In [10]:
targets = [d['target'] for d in data if 'UDGcand' in d['target']]
# targets

- Create an array of { name, properties } containing every object in the FITS file that's also a SpaceFluff object:

In [11]:
spacefluff_data = []

for d in data:
    if d['target'] in set(candidate_names_classify):
        
        object_properties = {
            "name": d[0]
        }
        
        for idx, column in enumerate(data.columns.names[1:]):
            object_properties[column] = d[idx+1]
        
        
        spacefluff_data.append(object_properties)

- Convert into a DataFrame, and inspect the head to see if nothing went wrong:

In [12]:
df_spacefluff_data = pd.DataFrame(spacefluff_data)

df_spacefluff_data.head()

,name,RA,DEC,Reff,r_mag,g_mag,axis_ratio,pos_angle,n,u,...,ge,re,ie,Reffe,r_mage,ne,C,mue_r,bae,RFF
0,UDGcand_0,56.232609,-35.335724,2.89340,19.831900,-1.0,0.6034,-27.775499,1.0271,22.426844,...,0.054816,0.046306,0.044244,0.332842,0.131324,0.117142,2.764210,23.585331,0.027318,-99.0
1,UDGcand_2,55.633900,-35.866928,2.63998,20.237101,-1.0,0.7488,15.694100,1.6470,25.864775,...,0.160716,0.051180,0.041536,0.378239,0.166238,0.140836,3.494850,24.025892,0.032711,-99.0
2,UDGcand_3,55.688835,-35.471954,1.97838,21.729500,-1.0,0.8003,10.619600,0.3000,23.426687,...,0.125680,0.112662,0.066605,0.452366,0.274637,0.208488,1.926754,24.964056,0.048007,-99.0
3,UDGcand_4,55.832767,-35.379551,5.79236,18.351999,-1.0,0.6255,61.521198,6.1206,22.914799,...,0.090910,0.037912,0.041479,0.688735,0.136073,0.120439,3.838429,23.651720,0.028070,-99.0
4,UDGcand_5,56.058018,-35.596409,2.98190,19.478001,-1.0,0.8144,-11.880000,0.8693,22.430893,...,0.055276,0.043606,0.043281,0.349424,0.133957,0.118973,2.764210,23.622435,0.027735,-99.0


- Save the DataFrame to csv for later use:

In [13]:
df_spacefluff_data.to_csv('./sf_spacefluff_object_data.csv', sep=",", index=False)

- Load and inspect created .csv to see if it saved correctly:

In [14]:
pd.read_csv('./sf_spacefluff_object_data.csv', comment="#").head()

,name,RA,DEC,Reff,r_mag,g_mag,axis_ratio,pos_angle,n,u,...,ge,re,ie,Reffe,r_mage,ne,C,mue_r,bae,RFF
0,UDGcand_0,56.232609,-35.335724,2.89340,19.831900,-1.0,0.6034,-27.775499,1.0271,22.426844,...,0.054816,0.046306,0.044244,0.332842,0.131324,0.117142,2.764210,23.585331,0.027318,-99.0
1,UDGcand_2,55.633900,-35.866928,2.63998,20.237101,-1.0,0.7488,15.694100,1.6470,25.864775,...,0.160716,0.051180,0.041536,0.378239,0.166238,0.140836,3.494850,24.025892,0.032711,-99.0
2,UDGcand_3,55.688835,-35.471954,1.97838,21.729500,-1.0,0.8003,10.619600,0.3000,23.426687,...,0.125680,0.112662,0.066605,0.452366,0.274637,0.208488,1.926754,24.964056,0.048007,-99.0
3,UDGcand_4,55.832767,-35.379551,5.79236,18.351999,-1.0,0.6255,61.521198,6.1206,22.914799,...,0.090910,0.037912,0.041479,0.688735,0.136073,0.120439,3.838429,23.651720,0.028070,-99.0
4,UDGcand_5,56.058018,-35.596409,2.98190,19.478001,-1.0,0.8144,-11.880000,0.8693,22.430893,...,0.055276,0.043606,0.043281,0.349424,0.133957,0.118973,2.764210,23.622435,0.027735,-99.0
